In [1]:
!pip  install --upgrade tables

Requirement already up-to-date: tables in /usr/local/lib/python3.6/dist-packages (3.6.1)


In [2]:




# connecting to google drive account
from google.colab import drive
drive.mount("/content/drive", force_remount=True)


Mounted at /content/drive


In [3]:
cd /content/drive/My\ Drive/Bert-Multi-Label-Text-Classification-master 

/content/drive/My Drive/Bert-Multi-Label-Text-Classification-master


In [0]:


!cp pybert/dataset/UC_datasets/Expanded_datasets/freq_data.hdf  ../UC_datasets/
!cp pybert/dataset/UC_datasets/Expanded_datasets/rare_data.hdf  ../UC_datasets/
!cp pybert/dataset/UC_datasets/Expanded_datasets/most_freq_data.hdf  ../UC_datasets/

In [0]:
cd /content/drive/My\ Drive/UC_datasets/pure_datasets

In [0]:
!ls


In [0]:
from sklearn.datasets import make_multilabel_classification

import torch
import os
import pandas as pd
import numpy as np
from collections import defaultdict
import random
from sklearn.model_selection import train_test_split

In [0]:
def IRLbl(y):
    LABELS = defaultdict(int)
    for sample in y:
        for label in sample:
            LABELS[label] += 1
            
    max_count = max(LABELS.values())
    for key,value in LABELS.items():
        # print(LABELS[key])
        LABELS[key] = max_count/LABELS[key]
        
    return LABELS

In [0]:

def MeanIR(y,ratios=False):
    imb_ratios = IRLbl(y)
    if ratios:
        return (sum(imb_ratios.values())/len(imb_ratios), imb_ratios)
    else:
        return (sum(imb_ratios.values())/len(imb_ratios))
        

MeanIR(y)

In [0]:
import numpy as np
def CVIR(y):
  mean_ir, ratios = MeanIR(y,ratios=True)
  ratios = np.array(list(ratios.values()))
  var = np.sqrt(sum((abs(ratios - mean_ir)**2))/(len(ratios)-1))
  # var = (sum((abs(ratios - mean_ir)))/(len(ratios)-1))

  cvir = var/mean_ir
  return cvir

In [0]:
# %debug
CVIR(y), CVIR(dataset[1])

NameError: ignored

In [0]:
def Label_Bags(X,y):
    " Constructs Bags of samples for every label in the dataset.\
    A label's Bag contains samples whose label set contains the label."
    assert(len(X) == len(y))
    Bags = defaultdict(list)
    for id, labels in enumerate(y):
        for label in labels:
            Bags[label].append([X[id],y[id]])
    return Bags

In [0]:

def ML_ROS(dataset, p):
    
    (X,y) = dataset
    X = list(X)
    y = list(y)

    samples_to_clone = len(X)*p/(100)
    mean_ir, imb_ratios = MeanIR(y=y, ratios=True)
    label_bags = Label_Bags(X,y)
    
    minBag = {}
    for label in label_bags.keys():
        if imb_ratios[label] > mean_ir:
            minBag[label] = label_bags[label]
    
    while samples_to_clone>0:
        if(minBag):
            for label in list(minBag):
                minBag_i = minBag[label]
                sample = random.sample(minBag_i,1)[0]

                minBag_i.append(sample)
                X.append(sample[0])
                y.append(sample[1])

                mean_ir, imb_ratios = MeanIR(y=y,ratios=True)
                if imb_ratios[label] <= mean_ir:
                    minBag.pop(label)
                    # print(f'label {label} popped')
                samples_to_clone-=1
                # print(f'{len(X)}, {samples_to_clone} mean_ir {mean_ir}')
        else:
            minBag = {}
            for label in label_bags.keys():
                if imb_ratios[label] > mean_ir:
                    minBag[label] = label_bags[label]
            
            if(minBag is None):
                break
    return X,y
                

In [22]:

X,y = make_multilabel_classification(n_classes=100,n_samples=1000,allow_unlabeled=False)

y = np.where(y==1)

y

y = pd.DataFrame({'row':y[0], 'col':y[1]})
y

y = y.groupby('row')['col'].apply(list)

list(y)
len(X),len(y)

(1000, 1000)

In [0]:
import pandas as pd

# data = pd.read_hdf('pybert/dataset/UC_datasets/Expanded_datasets/pure_datasets/all/data.hdf')
data = pd.read_hdf('pybert/dataset/UC_datasets/Expanded_datasets/Remedial/data.hdf')

In [0]:
X = data.symptoms
y = data.label

In [17]:
for i in [225,250, 275, 300]:
  dataset = ML_ROS((X,y),i)
  print(f'{MeanIR(dataset[1])} {CVIR(dataset[1])}')

11.85257861627801 0.4325758378623938
11.178483975101129 0.43894841355292585
11.647649811227337 0.45095739219576175
11.123389254129814 0.43932593142257054


In [16]:
for i in [125,150, 175, 200]:
  dataset = ML_ROS((X,y),i)
  print(f'{MeanIR(dataset[1])} {CVIR(dataset[1])}')

15.568632974100954 0.4908981943949905
13.688421315194645 0.5023222049268212
13.463431143374745 0.4717400351388647
13.321496908839523 0.5194735425747684


In [32]:
# dataset={}
# for i in [10,20,30,50,100]:
#   dataset[i] = ML_ROS((X,y),i)
  # print(f'{MeanIR(dataset[i][1])} {CVIR(dataset[i][1])}')
import pandas as pd

for key, value in dataset.items():
  d =  pd.DataFrame({'symptoms':dataset[key][0], 'label': dataset[key][1]} )
  d.to_hdf('../Bert-Multi-Label-Text-Classification-master/pybert/dataset/UC_datasets/Expanded_datasets/Remedial/oversampled/'+"oversampled_"+str(key), 'df')

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:2530: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->['symptoms', 'label']]

  pytables.to_hdf(path_or_buf, key, self, **kwargs)


In [20]:

for i in [10,20,30,50,100]:
  dataset = ML_ROS((X,y),i)
  print(f'{MeanIR(dataset[1])} {CVIR(dataset[1])}')

60.618176822305536 0.9595853456732205
38.55293742942471 0.63046579083383
31.7073724367306 0.623737528439836
24.857158073567625 0.5775221888322238
16.14558821026384 0.45479736032065127


In [12]:
MeanIR(y), CVIR(y)

(336.03885050570887, 2.3057385468468743)

In [0]:
# len(dataset[0])
  MeanIR(dataset[1],False)
# len(dataset[1])

In [0]:

len(X),len(y), len(dataset[0]), len(dataset[1])

In [0]:
# data_folder = os.path.join(os.pardir,'UC_datasets')
# data_folder = os.path.join(os.curdir,'pybert/dataset/train.csv')
# data_folder = os.curdir
data_folder = '../UC_datasets/pure_datasets/REMEDIAL/'

In [0]:
# data = pd.read_hdf(os.path.join(data_folder,'data.hdf'))
data = pd.read_hdf(os.path.join(data_folder,'data.hdf'))

In [0]:
MeanIR(data.label,True)

In [0]:

# data
# os.path.abspath(os.curdir)
# !mv data.hdff data.hdf
%ls

In [0]:
# files = os.listdir()
# files = [file for file in files if not os.path.isdir(file)]
# files

files = ['pybert/dataset/UC_datasets/Expanded_datasets/pure_datasets/all/data.hdf']

In [0]:
for file in files[0:1]:
  data = pd.read_hdf(file)
  X = data.symptoms
  y = data.label
  dataset_100 = ML_ROS((X,y), 100)
  dataset_50 = ML_ROS((X,y),50)
  dataset_30 = ML_ROS((X,y), 30)
  dataset_20 = ML_ROS((X,y), 20)
  dataset_10 = ML_ROS((X,y), 10)
  datasets = [dataset_10, dataset_20, dataset_30, dataset_50, dataset_100]
  ids = [10,20,30,50,100]
  for id,dataset in enumerate(datasets[:]):
    print(f'dataset_{str(ids[id])} {MeanIR(dataset[1])} {CVIR(dataset[1])}')
    X = dataset[0]
    y = dataset[1]

    X = np.stack(X)
    data  = pd.DataFrame({'symptoms':X, 'label':y})

    data = data.sample(frac=1)
    file = file.split('.')[0]
    # print(f'\n\n\nfile {file}\n\n\n')
    # data.to_hdf('over_sampled_'+file+"_"+str(ids[id])+'.hdf', 'df', format = 'fixed')
    data.to_hdf('over_sampled_'+'data'+"_"+str(ids[id])+'.hdf', 'df', format = 'fixed')

    train_data, test_data = train_test_split(data)
    # train_data.to_hdf('over_sampled_train_'+file+"_"+str(ids[id])+'.hdf', 'df', format = 'fixed')  
    # test_data.to_hdf('over_sampled_test_'+file+"_"+str(ids[id])+'.hdf', 'df', format = 'fixed')    
    train_data.to_hdf('over_sampled_train_'+'data'+"_"+str(ids[id])+'.hdf', 'df', format = 'fixed')  
    test_data.to_hdf('over_sampled_test_'+'data'+"_"+str(ids[id])+'.hdf', 'df', format = 'fixed')        

dataset_10 60.53423578718441 1.067827098069522


/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:2530: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->['symptoms', 'label']]

  pytables.to_hdf(path_or_buf, key, self, **kwargs)


dataset_20 37.283285226963535 0.7028239089021768
dataset_30 31.415518958451216 0.6209167235852044
dataset_50 24.444997673410388 0.660230428358072
dataset_100 17.66348838352838 0.4882560418635616


In [0]:
%debug

> /usr/local/lib/python3.6/dist-packages/tables/utils.py(172)check_file_access()
    170                 parentname = '.'
    171             if not os.access(parentname, os.F_OK):
--> 172                 raise IOError("``%s`` does not exist" % (parentname,))
    173             if not os.path.isdir(parentname):
    174                 raise IOError("``%s`` is not a directory" % (parentname,))

ipdb> file
*** NameError: name 'file' is not defined
ipdb> parentname
'over_sampled_pybert/dataset/UC_datasets/Expanded_datasets/pure_datasets/all'
ipdb> exit


In [0]:
data = pd.read_hdf('pybert/dataset/UC_datasets/Expanded_datasets/Remedial/data.hdf')
MeanIR(data.label)

336.03885050570887

In [0]:
X = data.symptoms

In [0]:

y = data.label

In [0]:
dataset = ML_ROS((X,y),50)

In [0]:
MeanIR(dataset[1]),CVIR(dataset[1])

(24.737415593381765, 0.5743337682664207)

In [0]:
dataset_30 = ML_ROS((X,y), 30)

In [0]:
dataset_20 = ML_ROS((X,y), 20)

In [0]:
dataset_100 = ML_ROS((X,y), 100)

In [0]:
dataset_50 = dataset

In [0]:
datasets = [dataset_20, dataset_30, dataset_50,dataset_100]
ids = [20,30,50,100]

In [0]:
# datasets = [dataset_100]
# ids = [100]

In [0]:
for id,dataset in enumerate(datasets[:]):
  X = dataset[0]
  y = dataset[1]

  X = np.stack(X)
  data  = pd.DataFrame({'symptoms':X, 'label':y})

  data = data.sample(frac=1)
  data.to_hdf('over_sampled_'+str(ids[id])+'.hdf', 'df', format = 'fixed')

  train_data, test_data = train_test_split(data)
  train_data.to_hdf('over_sampled_train_'+str(ids[id])+'.hdf', 'df', format = 'fixed')  
  test_data.to_hdf('over_sampled_test_'+str(ids[id])+'.hdf', 'df', format = 'fixed')  

In [0]:
!mv  over_sampled_* ../UC_datasets/pure_datasets/REMEDIAL/over_sampled

In [0]:
df = pd.read_hdf('../UC_datasets/pure_datasets/REMEDIAL/over_sampled/over_sampled_50.hdf')
# ds = ML_ROS((df.symptoms,df.label),50)
MeanIR(df.label)

17.095768597901973

In [0]:
!rm over_sampled_train0.hdf over_sampled_train_10.hdf

In [0]:

data = data.sample(frac=1)

In [0]:
oversampled_train_df = data.iloc[:20000]

In [0]:
oversampled_test_df = data.iloc[20000:]

In [0]:
oversampled_test_df.to_hdf('pybert/dataset/UC_datasets/oversampled_test.hdf','df',format='fixed')


In [0]:
oversampled_train_df.to_hdf('pybert/dataset/UC_datasets/oversampled_train.hdf','df')


In [0]:
X = data.symptoms
y = data.label

In [0]:
mean_ir, ratios = MeanIR(y, ratios=True)

In [0]:
(np.array(list(ratios.values())) > mean_ir).sum()

In [0]:
rare_med = [label  for label in ratios.keys() if ratios[label] > mean_ir]

In [0]:
rare_med, len(rare_med)

In [0]:
# dataset = ML_ROS((X,y),50)
# dataset=dataset_50
dataset = dataset_100

In [0]:
mean_ir_a, ratios_a = MeanIR(dataset[1],True)


In [0]:
rare_med_a = [label  for label in ratios_a.keys() if ratios_a[label] > mean_ir_a]

In [0]:
mean_ir_a, rare_med_a,len(rare_med_a)

In [0]:
sorted(list(ratios_a.values()))

In [0]:
len(dataset[0])

In [0]:
dataset[1]

In [0]:
X = np.stack(dataset[0])
y = dataset[1]

In [0]:
df = pd.DataFrame({'symptoms':X, 'label': y})

In [0]:
df

In [0]:
df.to_hdf('over_sampled.hdf','df',format='fixed')

In [0]:
import pandas as pd

df = pd.read_hdf('over_sampled.hdf','df',format='fixed')

In [0]:
LABELS = defaultdict(int)
for sample in dataset[1]:
    for label in sample:
        LABELS[label] += 1
        
max_count = max(LABELS.values())
print(max_count)
sum=0
for key,value in LABELS.items():
    sum+= value


for key,value in LABELS.items():
    LABELS[key] = LABELS[key]/max_count


In [0]:
sorted(list(LABELS.values()))
# LABELS

In [0]:
!cp pybert/dataset/UC_datasets/Expanded_datasets/over_sampled/over_sampled_train_50.hdf pybert/dataset/train.csv

In [0]:
!cp pybert/dataset/UC_datasets/Expanded_datasets/over_sampled/over_sampled_test_50.hdf pybert/dataset/test.csv

In [0]:
df = pd.read_hdf('over_sampled_train_10.hdf')

In [0]:
df

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
train_test_split(df)

In [0]:
a ,b = train_test_split(df.iloc[:100])

In [0]:
a

In [0]:
b

In [0]:
%ls


In [0]:
tr_data = pd.read_hdf('../Bert-Multi-Label-Text-Classification-master/pybert/dataset/UC_datasets/Expanded_datasets/over_sampled/over_sampled_train_100.hdf')
tr_data

In [0]:
tt_data = pd.read_hdf('../Bert-Multi-Label-Text-Classification-master/pybert/dataset/UC_datasets/Expanded_datasets/over_sampled/over_sampled_test_100.hdf')
tt_data

In [0]:
MeanIR(tr_data.label)

In [0]:
MeanIR(tt_data.label)

In [0]:
data = pd.read_hdf('../Bert-Multi-Label-Text-Classification-master/pybert/dataset/UC_datasets/Expanded_datasets/over_sampled/over_sampled_100.hdf')
data

In [0]:
MeanIR(data.label)

In [0]:
data = pd.read_hdf('pybert/dataset/train.csv')
data

In [0]:
MeanIR(data.label)

In [0]:
!cp  pybert/dataset/UC_datasets/Expanded_datasets/over_sampled/over_sampled_train_100.hdf pybert/dataset/train.csv


In [0]:
!cp  pybert/dataset/UC_datasets/Expanded_datasets/over_sampled/over_sampled_test_100.hdf pybert/dataset/test.csv


In [0]:
data = pd.read_hdf('pybert/dataset/train.csv')

In [0]:
labels = data.label

In [0]:
from collections import defaultdict

In [0]:
med_c = defaultdict(int)

In [0]:
for medicines in labels:
  for medicine in medicines:
    med_c[medicine]+=1

In [0]:
med_c

In [0]:
from sklearn.preprocessing import MultiLabelBinarizer

In [0]:
le = MultiLabelBinarizer()

In [0]:
label = le.fit_transform(labels)

In [0]:
l = []
for vec in label:
  l.append(np.where(vec ==1))

In [0]:
l

In [0]:
print(labels.to_string())


In [0]:
x = list(med_c.values())

In [0]:
x.sort(reverse=True)

In [0]:
x